FIAP - Redes Neurais Artificiais, Deep Learning e Algoritmos Genéticos 

# Aula 03 - Trabalhando com Perceptron

## Implementação de um Perceptron simples

In [ ]:
# Definir a função de agregação de informação (soma poderada)


In [ ]:
# Definir a função de ativação: FUNÇÃO DEGRAU


## Casos de aplicação do perceptron

Vamos analisar a base `escolha_faculdade.xlsx` com exemplos de faculdades identificadas pelo ID e com um conjunto de variáveis que as descrevem. As variáveis são: 

- reconhecida_MEC: a faculdade é reconhecida pelo MEC? 1=sim ; 0=não
- facilidade_pagamento: a faculdade apresenta facilidades para pagamento? 1=sim ; 0=não
- flexibilidade_ensino: o quão flexível é o método de ensino? 1=somente presencial ; 2=somente EAD ; 3=misto
- entrada_mercado: o que a faculdade oferece para facilitar a entrada do aluno no mercado de trabalho? 0=sem suporte ; 1=feiras ; 2=feiras e estágios ; 3=feiras, estágios e convênios com empresas
- score_infraestrutura: qual nota de 0 a 100 a escola possui para sua infraestrutura? 
- empregabilidade: qual a porcentagem de alunos egressos empregados? 
- acesso_facil: a localização possui fácil acesso, como metrô, ônibus, estacionamento? 1=sim ; 0=não

In [ ]:
# Pesos já fornecidos 
pesos = [ 0.10, 0.05, 0.05, 0.2, 0.25, 0.3, 0.05 ]

In [2]:
import pandas as pd
import math

In [ ]:
df = pd.read_excel("escolha_faculdade.xlsx", index_col="ID")